# Reading in and preparing data 

When running the notebooks on VITO’s servers, the user can follow the directions shown in 
https://nbviewer.jupyter.org/github/VITObelgium/notebook-samples/raw/master/datasets/probav/Reading%20PROBA-V%20using%20R.ipynb to find locations of PROBA-V products. Doing so, the data set (netCDF files) will be located and available in the VITO's servers.

Alternatively, when working locally, the R-user can choose to automatically download Copernicus land data (https://land.copernicus.eu/global/) using the functions found in https://github.com/cgls/Copernicus-Global-Land-Service-Data-Download-with-R. 

Once the data set is available, *raster* package functionality is used to prepare it for resampling. 

In [ ]:
library(raster)

# ndvi_files is a list of the available files (netCFD or Raster* objects)
ndvi_files <- list(paste0("/Users/xavi_rp/Documents/D6_LPD/NDVI_resample", "/ndvi300m_Cat.tif"))
r <- raster(ndvi_files[1])


The original netCDF file might contain flagged data corresponding to NoData (NA), etc, which needs to be dealt with. In case of the NDVI products and netCDF files, assigned values > 250 are flagged and need to be converted into NA. When netCDF files are read in as a *Raster*\* object and values transformed into real NDVI values, the cuttoff for flagged cells is 0.92. Therefore, all cells > 0.92 have to be set to NA.



In [ ]:
if(grepl("ndvi", ndvi_files[1], ignore.case = TRUE)){
  cuttoff_flag <- 0.92
}else{
  stop("Please define 'cuttoff_flag' (cuttoff value for flagged cells) for the Raster* object")
}

r[r > cuttoff_flag] <- NA


# Resampling using the aggregation approach

There are several approaches to resample data to a coarser resolution. The area-based aggregation methods are based in grouping rectangular areas of cells of the finer resolution image to create a new map with larger cells. To do that, the function *aggregate()* of the package *raster* needs to know the factor of aggregation. In this case, the factor is 3 as it needs to go from 333m to 1km. In addition, *aggregate()* can perform the calculation using different functions. While the default is the average (*mean()*) it can work also with *modal()*, *max()* or *min()*.

As for the pixel coordinate definition for both the 300m and the 1km products, no proper aggregation of
the finer resolution product can be performed at the minimum and maximum latitude and longitude[^1]. For this reason, the *RasterLayer* object needs to be 'cropped' accordingly.

[^1]: Erwin Wolters, Wouter Dierckx, Marian-Daniel Iordache, and Else Swinnen. (16/03/2018) PROBA-V Products User Manual v3.01. http://proba-v.vgt.vito.be/sites/proba-v.vgt.vito.be/files/products_user_manual.pdf


In [ ]:
if(extent(r)[1] < -180 & extent(r)[2] > 179.997){  # full image
  extnt_r <- extent(r)
  extnt_r[1] <- extent(r)[1] + (2 * (1 / 336)) # x-min
  extnt_r[2] <- extent(r)[2] - (1 * (1 / 336)) # x-max
  extnt_r[3] <- extent(r)[3] + (1 * (1 / 336))  # y-min
  extnt_r[4] <- extent(r)[4] - (2 * (1 / 336))  # y-max
  r <- crop(r, extnt_r)
}else{
  stop("If you have subset part of the image, make sure that your new extent coincides with the 1km product grid.
       (see http://proba-v.vgt.vito.be/sites/proba-v.vgt.vito.be/files/products_user_manual.pdf, pg 30). 
       If it does, go ahead with the resampling.")
}


If the user has subset part of the original 300m product, its new extention should match with the 1km product grid. Otherwise, the comparison or any other kind of calculations involving any of the 1km products cannot be made properly. The following vectors contain Long and Lat coordinates, respectively, of the 1km grid (edges of the cells):

In [ ]:

x_ext <- seq((-180 - ((1 / 112) / 2)), 180, (3 * (1 / 336)))
y_ext <- seq((80 + ((1 / 112) / 2)), - 60, - (3 * (1 / 336)))


When these checks and coorections are made, the process of resampling itself can go ahead using *aggregate()*.

In [ ]:
r300m_resampled1km_Aggr <- aggregate(r,
                                     fact = 3, # from 333m to 1km  
                                     fun = mean, 
                                     na.rm = TRUE, 
                                     filename = '')


Finally, a couple of plots in case the user wants to take a look at the resampled map.

In [ ]:
plot(r, main = 'Original map at 300m')
plot(r300m_resampled1km_Aggr, main = 'Resampled map to 1km')